<a href="https://colab.research.google.com/github/JoungMinJu/lab_study/blob/main/ELMo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ELMo 표현 사용해 스팸 메일 분류하기


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
! pip install tensorflow-hub

In [4]:
import tensorflow_hub as hub

In [6]:
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np


In [7]:
elmo=hub.Module('https://tfhub.dev/google/elmo/1', trainable=True)

#텐서플로우 허브로부터 ELMo를 다운로드

In [8]:
sess=tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [9]:
#스팸메일 분류하기 데이터 다운로드
urllib.request.urlretrieve('https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv', filename='spam.csv')
data=pd.read_csv('spam.csv',encoding='latin-1')
data[:5]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [10]:
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])  #v1숫자로 바꿈
y_data = list(data['v1'])
X_data = list(data['v2'])


In [11]:
print(X_data[:5])
print()
print()
print(y_data[:5])


['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...', "Nah I don't think he goes to usf, he lives around here though"]


[0, 0, 1, 0, 0]


In [12]:
#훈련데이터와 테스트 데이터 8:2로 분할
print(len(X_data))
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train)
print(n_of_test)

5572
4457
1115


In [13]:
X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

In [15]:
#ElMo와 설계한 모델을 연결하는 작업 짆애
#ELMo는 텐서플로우 허브로부터 가져온 것이므로 케라스에서 사용하기 위해서는 케라스에서 사용할 수 있도록 변환해주는 작업이 필요

def ELMoEmbedding(x):
  return elmo(tf.squeeze(tf.cast(x,tf.string)),as_dict=True, signature='default')['default']

#데이터의 이동이 케라스 -> 텐서플로우 -> 케라스가 되도록 하는 함수


In [18]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [19]:
#모델으 ELMo 이용한 임베딩 층을 거쳐서 256개의 뉴런이 있는 은닉층을 거친 후 
#마지막 한 개의 뉴런을 통해 이진분류를 수행

#이진분류를 위한 마지막 뉴런의 활성화 함수는 시그모이드 함수
#모델의 손실함수는 binary_crossentropy

history=model.fit(X_train, y_train, epochs=1, batch_size=60)

Epoch 1/1
4457/4457 [==============================] - 986s 221ms/step - loss: 0.1025 - accuracy: 0.9637


In [20]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

1115/1115 [==============================] - 249s 223ms/step

 테스트 정확도: 0.9785
